# 🔬 Semiconductor Defect Detection - Training Notebook

This notebook trains EfficientNet-B0 and MobileNetV3-Small models for semiconductor defect classification.

**Usage on Google Colab:**
1. Upload your dataset to Google Drive
2. Mount Drive and update DATA_DIR path
3. Run all cells

**Target:** 94-95% accuracy, <2MB model size

In [ ]:
# Install dependencies (Colab)
!pip install albumentations imagehash tf2onnx -q

In [ ]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, Model
from tensorflow.keras.applications import EfficientNetB0, MobileNetV3Small
import numpy as np
from pathlib import Path
from datetime import datetime

print(f"TensorFlow: {tf.__version__}")
print(f"GPUs: {tf.config.list_physical_devices('GPU')}")

## 📁 Configuration

In [ ]:
# UPDATE THIS PATH for your data location
DATA_DIR = Path('/content/drive/MyDrive/semiconductor_data/augmented')
# Or for local: DATA_DIR = Path('k:/iesa/data/augmented')

OUTPUT_DIR = Path('/content/outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

# Classes
CLASS_NAMES = [
    "clean", "scratches", "particles", "pattern_defects",
    "edge_defects", "center_defects", "random_defects", "other"
]
NUM_CLASSES = 8
IMAGE_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)

# Training
BATCH_SIZE = 32
STAGE1_EPOCHS = 15
STAGE2_EPOCHS = 10
STAGE1_LR = 1e-3
STAGE2_LR = 1e-5

## 📊 Data Loading

In [ ]:
def create_dataset(data_dir, split, batch_size=32, augment=False):
    """Create tf.data.Dataset from directory structure"""
    split_dir = data_dir / split
    
    image_paths = []
    labels = []
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        class_dir = split_dir / class_name
        if not class_dir.exists():
            continue
        for img_path in class_dir.glob("*.png"):
            image_paths.append(str(img_path))
            labels.append(class_idx)
    
    def parse_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.grayscale_to_rgb(img)
        img = tf.image.resize(img, IMAGE_SIZE)
        img = tf.cast(img, tf.float32) / 255.0
        return img, tf.one_hot(label, NUM_CLASSES)
    
    def augment_fn(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, 0.1)
        image = tf.clip_by_value(image, 0.0, 1.0)
        return image, label
    
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.shuffle(len(image_paths))
    ds = ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.cache()
    if augment:
        ds = ds.map(augment_fn, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# Load datasets
train_ds = create_dataset(DATA_DIR, 'train', BATCH_SIZE, augment=True)
val_ds = create_dataset(DATA_DIR, 'val', BATCH_SIZE)
test_ds = create_dataset(DATA_DIR, 'test', BATCH_SIZE)

print(f"Train: {len(list(train_ds))} batches")
print(f"Val: {len(list(val_ds))} batches")

## 🏗️ Model Creation

In [ ]:
def create_efficientnet(freeze_percent=0.8):
    """Create EfficientNet-B0 with custom head"""
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_image")
    
    base = EfficientNetB0(include_top=False, weights="imagenet", input_shape=INPUT_SHAPE)
    
    # Freeze layers
    freeze_until = int(len(base.layers) * freeze_percent)
    for i, layer in enumerate(base.layers):
        layer.trainable = i >= freeze_until
    
    x = base(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    return Model(inputs, outputs, name="efficientnet_classifier")

def create_mobilenet(freeze_percent=0.75):
    """Create MobileNetV3-Small with custom head"""
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_image")
    
    base = MobileNetV3Small(include_top=False, weights="imagenet", input_shape=INPUT_SHAPE)
    
    freeze_until = int(len(base.layers) * freeze_percent)
    for i, layer in enumerate(base.layers):
        layer.trainable = i >= freeze_until
    
    x = base(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(96, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    return Model(inputs, outputs, name="mobilenet_classifier")

# Create models
eff_model = create_efficientnet()
eff_model.summary()

## 🚀 Training

In [ ]:
def compile_model(model, lr):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
    )
    return model

def get_callbacks(name):
    return [
        keras.callbacks.ModelCheckpoint(
            f'{OUTPUT_DIR}/{name}_best.keras',
            monitor='val_accuracy', save_best_only=True
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy', patience=5, restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3
        )
    ]

In [ ]:
# Stage 1: Transfer Learning
print("=" * 50)
print("STAGE 1: Transfer Learning")
print("=" * 50)

eff_model = compile_model(eff_model, STAGE1_LR)

history1 = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=STAGE1_EPOCHS,
    callbacks=get_callbacks('efficientnet_s1')
)

print(f"\nStage 1 Best Val Accuracy: {max(history1.history['val_accuracy']):.4f}")

In [ ]:
# Stage 2: Fine-tuning (unfreeze more layers)
print("=" * 50)
print("STAGE 2: Fine-tuning")
print("=" * 50)

# Unfreeze more layers
base = eff_model.layers[1]  # EfficientNet base
for layer in base.layers[-50:]:
    layer.trainable = True

eff_model = compile_model(eff_model, STAGE2_LR)

history2 = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=STAGE2_EPOCHS,
    callbacks=get_callbacks('efficientnet_s2')
)

print(f"\nStage 2 Best Val Accuracy: {max(history2.history['val_accuracy']):.4f}")

In [ ]:
# Evaluate on test set
print("=" * 50)
print("EVALUATION")
print("=" * 50)

results = eff_model.evaluate(test_ds)
print(f"\nTest Accuracy: {results[1]:.4f}")
print(f"Test Precision: {results[2]:.4f}")
print(f"Test Recall: {results[3]:.4f}")

In [ ]:
# Save models
eff_model.save(f'{OUTPUT_DIR}/efficientnet_final.keras')
eff_model.save(f'{OUTPUT_DIR}/efficientnet_final.h5')
print("Models saved!")

## 📦 Export to ONNX

In [ ]:
# Export to ONNX (Phase 1 requirement)
import tf2onnx

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = f"{OUTPUT_DIR}/efficientnet_defect_classifier.onnx"

model_proto, _ = tf2onnx.convert.from_keras(eff_model, input_signature=spec, output_path=output_path)
print(f"ONNX model saved to: {output_path}")

In [ ]:
# Quantize to TFLite (for edge deployment)
converter = tf.lite.TFLiteConverter.from_keras_model(eff_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]

tflite_model = converter.convert()

tflite_path = f"{OUTPUT_DIR}/efficientnet_quantized.tflite"
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved: {tflite_path}")
print(f"Size: {len(tflite_model) / 1024 / 1024:.2f} MB")